In [1]:
from compartmentsBase import *
from sympy import *
init_printing()


# temp helpers 
def display_hc(k,g,pi,name='c'):
    lhs = symbols('k_' + name)
    rhs = k
    display(Eq(lhs,rhs,evaluate=False))

    lhs = symbols('g_' + name)
    rhs = g
    display(Eq(lhs,rhs,evaluate=False))

    lhs = symbols('\pi_' + name)
    rhs = pi
    display(Eq(lhs,rhs,evaluate=False))


$$
\def\n{\mathbf{n}}
\def\x{\mathbf{x}}
\def\N{\mathbb{\mathbb{N}}}
\def\X{\mathbb{X}}
\def\NX{\mathbb{\N_0^\X}}
\def\C{\mathcal{C}}
\def\Jc{\mathcal{J}_c}
\def\DM{\Delta M_{c,j}}
\newcommand\diff{\mathop{}\!\mathrm{d}}
\def\Xc{\mathbf{X}_c}
\newcommand{\muset}[1]{\dot{\{}#1\dot{\}}}
$$


For a compartment population $\n \in \NX$ evcolving stochastically according to stoichiometric equations from transition classes $\C$, we want to find an expression for
$$
\frac{\diff}{\diff t}\left< f(M^\gamma, M^{\gamma'}, \ldots) \right>
$$
in terms of expectations of population moments $M^\alpha, M^{\beta}, \ldots$

In [2]:
D = 1 # number of species

x = ContentVar('x')
y = ContentVar('y')
# Coagulation
transition_C = Transition(Compartment(x) + Compartment(y), Compartment(x + y))
k_C = Constant('k_C')
g_C = 1
pi_C = 1

transitions = [(transition_C, k_C, g_C, pi_C)]

display(transition_C)
display_hc(k_C, g_C, pi_C, name='C')

[x] + [y] ---> [x + y])

In [3]:
from ito import *
from compartmentsB import *

dM0 = ito(Moment(0))
display(dM0)

DeltaM(0)

In [4]:
monomials = decomposeMomentsPolynomial(dM0)

c = 0
q = 0

transition = transitions[c]
monomial = monomials[q]

display(transitions[c])
display((monomial.k, monomial.pM, monomial.pDM))

reactants = getCompartments(transition[0].lhs)
products = getCompartments(transition[0].rhs)

display(reactants)
display(products)

([x] + [y] ---> [x + y]), k_C, 1, 1)

(1, 1, DeltaM(0))

defaultdict(int, {Compartment(x): 1, Compartment(y): 1})

defaultdict(int, {Compartment(x + y): 1})

In [5]:
monomial = monomials[q]

(transition, k_c, g_c, pi_c) = transitions[c]
(k_q, pM, pDM) = (monomial.k, monomial.pM, monomial.pDM)

reactants = getCompartments(transition.lhs)
products = getCompartments(transition.rhs)

DM_cj = deltaM(reactants, products, D)
display(Eq(symbols('\\text{DM_cj}'), DM_cj))

pDMcj = subsDeltaM(pDM, DM_cj)
display(Eq(symbols('\\text{pDM}'), pDM))
display(Eq(symbols('\\text{pDMcj}'), pDMcj))

# TODO ...
def conditional_expectation(pDMcj, pi_c):
    return pDMcj # should be pi_c(pDMcj, ???)
   
cexp = conditional_expectation(pDMcj, pi_c)
l_n_Xc = k_c * k_q * pM * g_c * cexp
display(Eq(symbols('\\text{lnXc}'), l_n_Xc))

                            \gamma[0]       \gamma[0]       \gamma[0]
\text{DM_cj} = (x[0] + y[0])          - x[0]          - y[0]         

\text{pDM} = DeltaM(0)

In [6]:
dfMdt = get_dfMdt_contrib(reactants, l_n_Xc, D)
display(dfMdt)

            2                   
  k_C⋅Moment (0)   k_C⋅Moment(0)
- ────────────── + ─────────────
        2                2      

### (5)
Now let
$$
l(\n; \Xc) = k_c \cdot k_q \cdot \Pi(M^{\gamma^k}) \cdot g_c(\Xc) \cdot
        \left<
            \Pi \DM^{\gamma^l} \;\big|\; \Xc
        \right>
$$

Plugging in the concrete $\gamma^l$ and expanding, $l(\n; \Xc)$ is a polynomial in $\Xc$.

Monomials are of the form $k \x^\alpha$ or $k \x^\alpha \x'^\beta$ with $\alpha, \beta \in \N_0^D$.
(Note that occurences of $\Pi M^{\gamma^k}$ are part of the constants $k$.)

Consider again the different cases of reactant compartments $\Xc$:

(1) $\Xc = \emptyset$:
$$
\frac{\diff\left< f(M^\gamma, M^{\gamma'}, \ldots) \right>}{\diff t}
    = \left<l(\n)\right>
$$

(2) $\Xc = \muset{\x}$:
$$
\frac{\diff\left< f(M^\gamma, M^{\gamma'}, \ldots) \right>}{\diff t}
    = \left<R(l(\n; \muset{\x})\right>
$$
where $R$ replaces all $k \x^\alpha$ by $k M^\alpha$.

(3) $\Xc = \muset{\x, \x'}$:
$$
\frac{\diff\left< f(M^\gamma, M^{\gamma'}, \ldots) \right>}{\diff t}
    = \frac{1}{2}\left<R'(l(\n; \muset{\x, \x'})\right>
    \: - \:
    \frac{1}{2}\left<R(l(\n; \muset{\x, \x})\right>
$$
where $R'$ replaces all $k \x^\alpha \X'^\beta$ by $k M^\alpha M^\beta$,
and again $R$ replaces all $k \x^\alpha$ by $k M^\alpha$.




### (6)
Finally, sum over contributions from all $c$, $q$ for the total
$$
\frac{\diff\left< f(M^\gamma, M^{\gamma'}, \ldots) \right>}{\diff t}
$$

In [7]:
from ito import *

def get_dfMdt(transitions, fM, D):
    dfM = ito(fM)
    monomials = decomposeMomentsPolynomial(dfM)
    contrib = list()
    for c, (transition, k_c, g_c, pi_c) in enumerate(transitions):
        for q, (k_q, pM, pDM) in enumerate([(m.k, m.pM, m.pDM) for m in monomials]):
            reactants = getCompartments(transition.lhs)
            products = getCompartments(transition.rhs)
            DM_cj = deltaM(reactants, products, D)
            pDMcj = subsDeltaM(pDM, DM_cj)
            cexp = conditional_expectation(pDMcj, pi_c)
            l_n_Xc = k_c * k_q * pM * g_c * cexp
            dfMdt = get_dfMdt_contrib(reactants, l_n_Xc, D)
            contrib.append(dfMdt)
            #print(f'c={c}, q={q}')
            #display(Eq(symbols('\\text{dfMdt}'), dfMdt))
    return Add(*contrib)

display(get_dfMdt(transitions, Moment(0), D))

            2                   
  k_C⋅Moment (0)   k_C⋅Moment(0)
- ────────────── + ─────────────
        2                2      